In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [13]:
import csv
import cv2
import mediapipe as mp  
import math
import os

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = tuple(a)
    b = tuple(b)
    c = tuple(c)

    radians = math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    angle = abs(math.degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Initialize Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Input folder containing video files
input_folder = 'A:\gym_tracker-ai\gym_tracker-ai\Diagonal Left and Right Dumbell Shoulder Press Test #1\Diag Right'
output_csv = 'Diagonal Right Dumbell Shoulder Press.csv'

# Open the CSV file to write the angles
with open(output_csv, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Write header row
    header = [f'Angle_{i}' for i in [11, 12, 13, 14, 23, 24, 25, 26]]
    csv_writer.writerow(header)

    # Process each video in the folder
    for video_file in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_file)
        if not video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue

        print(f"Processing video: {video_file}")

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Recolor image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make detection
                results = pose.process(image)

                # Recolor back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract landmarks and calculate angles
                try:
                    if results.pose_landmarks:
                        landmarks = results.pose_landmarks.landmark
                        angles = []

                        # Extract required points for angle calculation
                        def get_coords(idx):
                            return [landmarks[idx].x, landmarks[idx].y]

                        # Define angle mapping
                        angle_map = {
                            11: (23, 11, 13),
                            12: (24, 12, 14),
                            13: (11, 13, 15),
                            14: (12, 14, 16),
                            23: (25, 23, 24),
                            24: (26, 24, 23),
                            25: (27, 25, 23),
                            26: (28, 26, 24)
                        }

                        for i, (a, b, c) in angle_map.items():
                            angle = calculate_angle(get_coords(a), get_coords(b), get_coords(c))
                            angles.append(angle)

                        # Write angles to CSV
                        csv_writer.writerow([video_file] + angles)

                except Exception as e:
                    print(f"Error processing frame: {e}")

                # Render detections
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )

                # Show the frame (optional, can be commented out for faster processing)
                cv2.imshow('Mediapipe Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()

    cv2.destroyAllWindows()

print(f"Pose angles saved to {output_csv}")


Processing video: Nested Sequence 01.mp4
Processing video: Nested Sequence 02.mp4
Processing video: Nested Sequence 03.mp4
Processing video: Nested Sequence 04.mp4
Processing video: Nested Sequence 05.mp4
Processing video: Nested Sequence 06.mp4
Processing video: Nested Sequence 07.mp4
Processing video: Nested Sequence 08.mp4
Processing video: Nested Sequence 09.mp4
Processing video: Nested Sequence 10.mp4
Pose angles saved to Diagonal Right Dumbell Shoulder Press.csv


<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >